`chain_samples()` function in [cDNA_Cupcake](https://github.com/Magdoll/cDNA_Cupcake) crashes with an out-of-memory error on some of my samples.  Try as I might, I've not been able to decode exactly what and why it does what it does (why does it write *everything* to a file and read it back in?).  However, I found that the [pyranges](https://github.com/biocore-ntnu/pyranges) module has a `cluster` function that will give a common id to overlapping sequences.  Need to see if that will work.

In [1]:
import pyranges as pr
import pandas as pd
from pathlib import Path

In [34]:
bc1001_gff = Path("test_data/bc1001/repaired.sample.gff")
bc1002_gff = Path("test_data/bc1002/repaired.sample.gff")
gencode_v37_gff = Path("test_data/gencode.v37.annotation.gtf.gz")

In [35]:
bc1001_annot = pr.read_gff(bc1001_gff)

In [36]:
bc1001_annot

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,transcript_id
0,chr1,PacBio,transcript,1389718,1394777,.,+,.,PB.3,PB.3.1
1,chr1,PacBio,exon,1389718,1390368,.,+,.,PB.3,PB.3.1
2,chr1,PacBio,exon,1390455,1390565,.,+,.,PB.3,PB.3.1
3,chr1,PacBio,exon,1390767,1394777,.,+,.,PB.3,PB.3.1
4,chr1,PacBio,transcript,2133700,2139625,.,+,.,PB.7,PB.7.1
...,...,...,...,...,...,...,...,...,...,...
208497,chrX,PacBio,exon,154789953,154790022,.,-,.,PB.3323,PB.3323.1
208498,chrX,PacBio,exon,154790982,154791068,.,-,.,PB.3323,PB.3323.1
208499,chrX,PacBio,exon,154791768,154791847,.,-,.,PB.3323,PB.3323.1
208500,chrX,PacBio,exon,154792141,154792285,.,-,.,PB.3323,PB.3323.1


In [37]:
bc1002_annot = pr.read_gff(bc1002_gff)
# gencode = pr.read_gff(gencode_v37_gff)

In [6]:
# gencode

In [7]:
# gencode.subset(lambda x: x["Feature"].isin(("transcript", "exon")))

In [8]:
# gencode_expressed = gencode.subset(lambda x: x["Feature"].isin(("transcript", "exon")))

In [38]:
bc1001_sample_name = pd.Series(["bc1001" for _ in range(len(bc1001_annot))], name="sample_source")
bc1001_annot = bc1001_annot.insert(bc1001_sample_name)

In [39]:
bc1001_annot.head()

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,transcript_id,sample_source
0,chr1,PacBio,transcript,1389718,1394777,.,+,.,PB.3,PB.3.1,bc1001
1,chr1,PacBio,exon,1389718,1390368,.,+,.,PB.3,PB.3.1,bc1001
2,chr1,PacBio,exon,1390455,1390565,.,+,.,PB.3,PB.3.1,bc1001
3,chr1,PacBio,exon,1390767,1394777,.,+,.,PB.3,PB.3.1,bc1001
4,chr1,PacBio,transcript,2133700,2139625,.,+,.,PB.7,PB.7.1,bc1001
5,chr1,PacBio,exon,2133700,2139625,.,+,.,PB.7,PB.7.1,bc1001
6,chr1,PacBio,transcript,2245074,2280214,.,+,.,PB.8,PB.8.1,bc1001
7,chr1,PacBio,exon,2245074,2245959,.,+,.,PB.8,PB.8.1,bc1001


In [40]:
bc1002_sample_name = pd.Series(["bc1002" for _ in range(len(bc1002_annot))], name="sample_source")
bc1002_annot = bc1002_annot.insert(bc1002_sample_name)

In [41]:
bc1001_1002 = bc1001_annot.join(bc1002_annot, how="left", nb_cpu=4)

2021-04-30 15:37:30,790	INFO services.py:1171 -- View the Ray dashboard at http://127.0.0.1:8265


Note: one issue here is that `join()` has to perform some sort of table join, it won't simply bind rows.  This necessarily means we are going to lose samples that are not present in both PyRanges objects. 

We could probably deal with this by performing multiple joins or, if nothing else, one can probably get around this by taking all the underlying dataframes, merging those together, converting to a `dict`, and then converting that back into a `PyRanges` object

In [42]:
bc1001_1002.head()

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,transcript_id,...,Source_b,Feature_b,Start_b,End_b,Score_b,Strand_b,Frame_b,gene_id_b,transcript_id_b,sample_source_b
0,chr1,PacBio,transcript,1389718,1394777,.,+,.,PB.3,PB.3.1,...,PacBio,transcript,1385709,1399339,.,-,.,PB.4,PB.4.1,bc1002
1,chr1,PacBio,transcript,1389718,1394777,.,+,.,PB.3,PB.3.1,...,PacBio,transcript,1386500,1399317,.,-,.,PB.4,PB.4.2,bc1002
2,chr1,PacBio,transcript,1389718,1394777,.,+,.,PB.3,PB.3.1,...,PacBio,transcript,1386504,1399338,.,-,.,PB.4,PB.4.3,bc1002
3,chr1,PacBio,transcript,1389718,1394777,.,+,.,PB.3,PB.3.1,...,PacBio,transcript,1386705,1399338,.,-,.,PB.4,PB.4.4,bc1002
4,chr1,PacBio,transcript,1389718,1394777,.,+,.,PB.3,PB.3.1,...,PacBio,transcript,1386709,1399328,.,-,.,PB.4,PB.4.5,bc1002
5,chr1,PacBio,transcript,1389718,1394777,.,+,.,PB.3,PB.3.1,...,PacBio,transcript,1386710,1399335,.,-,.,PB.4,PB.4.7,bc1002
6,chr1,PacBio,transcript,1389718,1394777,.,+,.,PB.3,PB.3.1,...,PacBio,transcript,1386710,1399317,.,-,.,PB.4,PB.4.6,bc1002
7,chr1,PacBio,transcript,1389718,1394777,.,+,.,PB.3,PB.3.1,...,PacBio,transcript,1387343,1399317,.,-,.,PB.4,PB.4.8,bc1002


In [43]:
bc1001_1002.cluster().subset(lambda x: x.Cluster == 1)

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,transcript_id,...,Feature_b,Start_b,End_b,Score_b,Strand_b,Frame_b,gene_id_b,transcript_id_b,sample_source_b,Cluster
0,chr1,PacBio,transcript,1389718,1394777,.,+,.,PB.3,PB.3.1,...,transcript,1385709,1399339,.,-,.,PB.4,PB.4.1,bc1002,1
1,chr1,PacBio,transcript,1389718,1394777,.,+,.,PB.3,PB.3.1,...,exon,1390765,1392803,.,-,.,PB.4,PB.4.8,bc1002,1
2,chr1,PacBio,transcript,1389718,1394777,.,+,.,PB.3,PB.3.1,...,exon,1390765,1392803,.,-,.,PB.4,PB.4.9,bc1002,1
3,chr1,PacBio,transcript,1389718,1394777,.,+,.,PB.3,PB.3.1,...,exon,1393395,1393460,.,-,.,PB.4,PB.4.1,bc1002,1
4,chr1,PacBio,transcript,1389718,1394777,.,+,.,PB.3,PB.3.1,...,exon,1393395,1393460,.,-,.,PB.4,PB.4.2,bc1002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,chr1,PacBio,exon,1390767,1394777,.,+,.,PB.3,PB.3.1,...,transcript,1387343,1399317,.,-,.,PB.4,PB.4.8,bc1002,1
104,chr1,PacBio,exon,1390767,1394777,.,+,.,PB.3,PB.3.1,...,transcript,1390334,1399339,.,-,.,PB.4,PB.4.9,bc1002,1
105,chr1,PacBio,exon,1390767,1394777,.,+,.,PB.3,PB.3.1,...,exon,1390765,1392803,.,-,.,PB.4,PB.4.1,bc1002,1
106,chr1,PacBio,exon,1390767,1394777,.,+,.,PB.3,PB.3.1,...,exon,1390765,1392803,.,-,.,PB.4,PB.4.2,bc1002,1


In [23]:
len(bc1001_1002)

5311945

In [24]:
len(bc1001_annot)

208502

In [25]:
len(bc1002_annot)

172893

Not sure why there are ~20-50 times as many rows in the merged set as there are in the two parental non-merged objects.

Another way of doing this would be to concatinate the files and then import and cluster.

In [26]:
bc1001_annot.to_gff3(path="alt_bc1001.gff3")

In [31]:
bc1002_annot.to_gff3(path="alt_bc1002.gff3")

In [35]:
bc1001_sample_name = pd.Series(["bc1001" for _ in range(len(bc1001_annot))], name="Source")

In [39]:
bc1001_annot.drop("sample_source").insert(bc1001_sample_name).to_gff3(path="alt_bc1001.gff3")

In [40]:
bc1002_sample_name = pd.Series(["bc1002" for _ in range(len(bc1002_annot))], name="Source")

In [41]:
bc1002_annot.drop("sample_source").insert(bc1002_sample_name).to_gff3(path="alt_bc1002.gff3")

In [42]:
!cat alt_bc1001.gff3 alt_bc1002.gff3 > bc1001-bc1002-merged.gff3

In [3]:
merged = pr.read_gtf("/home/milo/workspace/cdna_brownie/notebooks/test_data/all-merged.gff")

Some of the clusters appear to have **very** cover very wide intervals, to the point where I'm not conviced these should "cluster".

For instance, bc1001/PB.13 bc1002/PB.7:

In [30]:
merged.subset(lambda x: x.Source == "bc1004").subset(lambda y: y.gene_id == "PB.13")

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,transcript_id,Cluster,Count
0,chr1,bc1004,transcript,2556370,2563747,.,+,.,PB.13,PB.13.2,2,44
1,chr1,bc1004,transcript,2555793,4508611,.,+,.,PB.13,PB.13.1,2,44


In [32]:
merged.subset(lambda x: x.Source == "bc1019").subset(lambda y: y.gene_id == "PB.7")

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,transcript_id,Cluster,Count
5,chr1,bc1019,transcript,3857266,3885426,.,+,.,PB.7,PB.7.1,2,44


This makes a bit more sense now: bc1004 PB.13.2 is a subset of PB.13.1 and bc1019 PB.7.1 is a subset of bc1004 PB.13.2

So, to summarize what we need to do:

1) In each GFF, change the "Source" field to the sample name.
2) Concatinate all GFFs.
3) Cluster the transcripts/exons in the merged PyRanges object.
4) Create a new PBID based on the cluster number and use this to convert the existing gene_id and transcript_ids.
5) For each distinct "Source", we need to create a translation table mapping the previous gene_is/transcript_id and the new cluster-based ones.
6) Convert the ids in the other associated files (i.e. the abundance and group files) for each sample

In [46]:
prdf = merged.as_df()

In [48]:
pr_dict = prdf.to_dict()

In [50]:
pd.DataFrame(data = pr_dict)

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,transcript_id,Cluster,Count
0,chr1,bc1004,transcript,833846,838936,.,+,.,PB.3,PB.3.1,1,4
1,chr1,bc1003,transcript,827658,857173,.,+,.,PB.2,PB.2.1,1,4
2,chr1,bc1002,transcript,854801,859446,.,+,.,PB.2,PB.2.1,1,4
3,chr1,bc1005,transcript,834239,838441,.,+,.,PB.1,PB.1.1,1,4
4,chr1,bc1004,transcript,2556370,2563747,.,+,.,PB.13,PB.13.2,2,44
...,...,...,...,...,...,...,...,...,...,...,...,...
175277,chrY,bc1012,transcript,19684562,19744726,.,-,.,PB.3466,PB.3466.1,3836,5
175278,chrY,bc1012,transcript,19684566,19744723,.,-,.,PB.3466,PB.3466.2,3836,5
175279,chrY,bc1012,transcript,19705423,19744726,.,-,.,PB.3466,PB.3466.3,3836,5
175280,chrY,bc1012,transcript,19705425,19744723,.,-,.,PB.3466,PB.3466.4,3836,5


In [52]:
pr.from_dict(pr_dict)

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,transcript_id,Cluster,Count
0,chr1,bc1004,transcript,833846,838936,.,+,.,PB.3,PB.3.1,1,4
1,chr1,bc1003,transcript,827658,857173,.,+,.,PB.2,PB.2.1,1,4
2,chr1,bc1002,transcript,854801,859446,.,+,.,PB.2,PB.2.1,1,4
3,chr1,bc1005,transcript,834239,838441,.,+,.,PB.1,PB.1.1,1,4
4,chr1,bc1004,transcript,2556370,2563747,.,+,.,PB.13,PB.13.2,2,44
...,...,...,...,...,...,...,...,...,...,...,...,...
175277,chrY,bc1012,transcript,19684562,19744726,.,-,.,PB.3466,PB.3466.1,3836,5
175278,chrY,bc1012,transcript,19684566,19744723,.,-,.,PB.3466,PB.3466.2,3836,5
175279,chrY,bc1012,transcript,19705423,19744726,.,-,.,PB.3466,PB.3466.3,3836,5
175280,chrY,bc1012,transcript,19705425,19744723,.,-,.,PB.3466,PB.3466.4,3836,5


In [54]:
merged2 = merged.copy()

In [62]:
merged2 = merged2.insert(pd.Series(["stuff" for _ in range(len(merged2.as_df()))], name="Source"))
merged2.head()

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,transcript_id,Cluster,Count
0,chr1,stuff,transcript,833846,838936,.,+,.,PB.3,PB.3.1,1,4
1,chr1,stuff,transcript,827658,857173,.,+,.,PB.2,PB.2.1,1,4
2,chr1,stuff,transcript,854801,859446,.,+,.,PB.2,PB.2.1,1,4
3,chr1,stuff,transcript,834239,838441,.,+,.,PB.1,PB.1.1,1,4
4,chr1,stuff,transcript,2556370,2563747,.,+,.,PB.13,PB.13.2,2,44
5,chr1,stuff,transcript,3857266,3885426,.,+,.,PB.7,PB.7.1,2,44
6,chr1,stuff,transcript,3857266,3885424,.,+,.,PB.12,PB.12.1,2,44
7,chr1,stuff,transcript,2556213,2563742,.,+,.,PB.9,PB.9.1,2,44


In [ ]:
merged2.insert()